In [14]:
from gurobipy import *

In [8]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\lifen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
from gurobipy import GRB
import gurobipy as gb

In [10]:
import pandas as pd

In [11]:
# Create the model
model = gb.Model("FIBA")

# Read the dataset
df = pd.read_csv(r'D:\2.OMIS6000 Models&application\asg2\BasketballPlayers.csv', index_col='Number')

# Create decision variables
x = model.addVars(150, vtype=gb.GRB.BINARY, name="Player")

# Set the objective function
model.setObjective(gb.quicksum(df.iloc[i, 1:8].sum() * x[i] for i in range(150)), gb.GRB.MAXIMIZE)

# Capacity constraint
model.addConstr(gb.quicksum(x[i] for i in range(150)) == 21, "Total Players")

# Position constraints
guard_indices = df[(df['Position'] == 'G') | (df['Position'] == 'G/F')].index.tolist()
model.addConstr(gb.quicksum(x[i] for i in guard_indices) >= 0.3 * 21, "Guard Position")

forward_center_indices = df[(df['Position'] == 'F') | (df['Position'] == 'C') | (df['Position'] == 'F/C')].index.tolist()
if forward_center_indices:
    model.addConstr(gb.quicksum(x[i] for i in forward_center_indices if i < 150) >= 0.4 * 21, "Forward/Center Position")

# Average score constraints
for j in range(7):
    model.addConstr((gb.quicksum(df.iloc[i, j+1] * x[i] for i in range(150)) / 21) >= 2.05, f"Average Score {j}")

# Constraint: if any 20-24, not all 72-78
for i in range(19, 24):
    model.addConstr(x[i] <= 1 - gb.quicksum(x[j] for j in range(71, 78)), f"If any 20-24, not all 72-78")

# Constraint: if any 105-114, at least one 45-49 and at least one 60-69
for i in range(104, 114):
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(44, 49)), f"If any 105-114, at least 45-49")
    model.addConstr(x[i] <= gb.quicksum(x[j] for j in range(64, 69)), f"If any 105-114, at least 65-69")

# We need at least one player from every 10 players
for i in range(14):
    model.addConstr(gb.quicksum(x[i*10 + j] for j in range(10)) >= 1, f"At least one player from group {i}")

# Optimize the model
model.optimize()

# Print the objective and decision variables
model.printAttr('X')

# Print the model status
print("Model Status: ", model.status)

# Count and print the number of guards invited
selected = [i for i, var in enumerate(model.getVars()) if var.x != 0]
guards_count = sum(1 for i in selected if df.iloc[i]['Position'] in ['G', 'G/F'])
print(f'Number of G and G/F: {guards_count}')


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 49 rows, 150 columns and 1649 nonzeros
Model fingerprint: 0xcbe95b5d
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Presolved: 49 rows, 150 columns, 1310 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.600000e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     360.0000000  360.00000  0.00%     -    0s

Explored 1 n

In [13]:
#2h:
# Read the data
ability_url = 'https://drive.google.com/uc?id=1_vOXTRV68lrCs_Q1c0zcQ21Q_q8AT6FK&export=download?usp=sharing'
ability = pd.read_csv(ability_url, index_col='Number')

def optimize_smallest_invitations():
    # Define the function to create the selection model
    def create_selection_model(number):
        # Create the optimization model
        model = gb.Model("FIBA Selection")

        # Create decision variables
        x = model.addVars(len(df), vtype=GRB.BINARY, name="Player")

        # Objective function
        model.setObjective(sum(df.iloc[i, 1:8].sum() * x[i] for i in range(len(df))), GRB.MAXIMIZE)

        # Add constraints
        model.addConstr(sum(x[i] for i in range(len(df))) == number, "Capacity Constraint")

        # Position constraints
        guard_indices = df[df['Position'].isin(['G', 'G/F'])].index
        model.addConstr(sum(x[i - 1] for i in guard_indices) >= number * 0.3, "Guard Position")
        forward_center_indices = df[df['Position'].isin(['F', 'C', 'F/C'])].index
        model.addConstr(sum(x[i - 1] for i in forward_center_indices) >= number * 0.4, "Forward/Center Position")

        # Average score constraints
        for j in range(7):
            model.addConstr(sum(x[i] * df.iloc[i, j + 1] for i in range(len(df))) / number >= 2.05, f"Average Score {j}")

        # Additional constraints
        for i in range(19, 24):
            for j in range(71, 78):
                model.addConstr(x[i] <= 1 - x[j], "Constraint 1")

        for i in range(104, 114):
            model.addConstr(x[i] <= sum(x[j] for j in range(44, 49)), "Constraint 2")
            model.addConstr(x[i] <= sum(x[j] for j in range(64, 69)), "Constraint 3")

        for i in range(14):
            model.addConstr(sum(x[i * 10 + j] for j in range(10)) >= 1, "At Least One Player")

        return model

    # Initialize the smallest number of invitations
    number_of_invitations = 21

    # Iterate to find the smallest number of invitations before infeasibility
    while True:
        # Set the number of invitations to the current value
        model = create_selection_model(number_of_invitations)

        # Solve the model
        model.optimize()

        # Check feasibility status
        if model.status != GRB.Status.OPTIMAL:
            # Infeasible solution found, break the loop
            break

        # Decrement the number of invitations for the next iteration
        number_of_invitations -= 1

    # Print the result
    print("Smallest number of invitations before infeasibility:", number_of_invitations + 1)

# Call the function to optimize smallest invitations
optimize_smallest_invitations()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 79 rows, 150 columns and 1680 nonzeros
Model fingerprint: 0x766b47ce
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Presolved: 79 rows, 150 columns, 1341 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.610000e+02, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     361.0000000  361.00000  0.00%     -    0s

Explored 1 n